In [1]:
# --- 安装必要的库 ---
# 请确保您已按照上述说明安装了 libvips 系统库
# pip install pyvips opencv-python scikit-image tqdm

import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.color import rgb2hed
from skimage.filters import threshold_otsu
from IPython.display import display, Image as IPImage

# # 导入 pyvips 库
# try:
#     import pyvips
# except ImportError:
#     print("错误: pyvips 库未找到。请按照单元格上方的说明进行安装。")
#     # 在 Notebook 中，我们可以用 raise 来停止执行
#     raise

# 使用 tqdm.notebook 可以在 Notebook 中显示更好看的进度条
from tqdm.auto import tqdm

os.chdir("/home1/jijh/diffusion_project/ADiffusion")
os.getcwd()


'/public/home/jijh/diffusion_project/ADiffusion'

In [9]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from scipy.sparse import issparse

# 导入您自定义的库
from src.pipeline.hest_loading import HESTDataset

def load_sample_data(sample, batch_key: str = 'sample_id'):
    """Helper function to load a single sample's data"""
    try:
        sample.load_st_data(lazy=False)
        if sample.adata is not None:
            # 确保每个adata有唯一的obs_names以避免冲突
            sample.adata.obs_names = [f"{sample.sample_id}_{obs_name}" for obs_name in sample.adata.obs_names]
            # 添加批次信息
            sample.adata.obs[batch_key] = sample.sample_id
            return sample.adata.to_memory()
        return None
    except Exception as e:
        print(f"Error loading sample {sample.sample_id}: {e}")
        return None

# --- 函数定义 ---

def calculate_global_hvgs(
    samples_to_process: list, 
    n_top_genes: int = 5000, 
    batch_key: str = 'sample_id',
    sentence_output_basedir_hvg: str = "output/hvgs"
    
) -> pd.Index:
    """
    合并多个AnnData对象，进行批次校正，并计算全局高变基因。
    (版本已修改，增强了过滤和HVG计算的稳定性)
    """
    print("开始合并所有样本以计算全局HVGs...")
    
    adatas = []
    for sample in tqdm(samples_to_process, desc="加载并合并AnnData"):
        adata = load_sample_data(sample, batch_key)
        if adata is not None:
            adatas.append(adata)

    if not adatas:
        raise ValueError("未能加载任何AnnData对象。")

    processed_adatas = []
    for adata in adatas:
        if adata.var_names.duplicated().any():
            print(f"发现重复基因名，正在处理样本中的 {adata.var_names.duplicated().sum()} 个重复基因...")
            adata = adata[:, ~adata.var_names.duplicated()].copy()
        processed_adatas.append(adata)
    
    adata_merged = sc.concat(processed_adatas, join='outer', uns_merge='unique')
    print(f"所有样本已合并。总尺寸: {adata_merged.shape}")
    
    print("开始进行SOTA质量控制和过滤...")
    
    adata_merged.var['mt'] = (
        adata_merged.var_names.str.startswith('MT-') |
        adata_merged.var_names.str.startswith('mt-')
    )
    adata_merged.var['ribo'] = (
        adata_merged.var_names.str.startswith(('RPS', 'RPL')) |
        adata_merged.var_names.str.startswith(('Rps', 'Rpl'))
    )
    adata_merged.var['hb'] = (
        adata_merged.var_names.str.contains('^HB[^(p)]', case=False) |
        adata_merged.var_names.str.contains('^Hb[^(p)]')
    )
    
    sc.pp.calculate_qc_metrics(adata_merged, qc_vars=['mt', 'ribo', 'hb'], percent_top=None, log1p=False, inplace=True)
    
    print(f"过滤前: {adata_merged.n_obs} spots, {adata_merged.n_vars} genes")
    
    sc.pp.filter_cells(adata_merged, min_genes=200)
    print(f"过滤低表达基因数spots后: {adata_merged.n_obs} spots")
    
    adata_merged = adata_merged[adata_merged.obs.pct_counts_mt < 20, :]
    print(f"过滤高线粒体比例spots后: {adata_merged.n_obs} spots")
    
    # --- 【修改点 1: 加强基因过滤】 ---
    # 对于百万级spot数据集，过滤掉在极少数spot中表达的基因
    min_cells_threshold = int(adata_merged.n_obs * 0.0005) # 至少在 0.05% 的 spot 中表达
    print(f"增强过滤: 基因必须在至少 {min_cells_threshold} 个 spots 中表达。")
    sc.pp.filter_genes(adata_merged, min_cells=min_cells_threshold)
    print(f"过滤低表达基因后: {adata_merged.n_vars} genes")
    
    print("进行标准化和对数变换...")
    sc.pp.normalize_total(adata_merged, target_sum=1e4)
    sc.pp.log1p(adata_merged)
    
    # --- 【修改点 2: 使用更稳健的HVG计算方法】 ---
    print("计算高变基因 (使用 'seurat_v3' flavor)...")
    sc.pp.highly_variable_genes(
        adata_merged, 
        n_top_genes=n_top_genes, 
        batch_key=batch_key, 
        flavor='seurat_v3'  # 使用更稳健的方法
    )
    
    hvg_list = adata_merged.var.index[adata_merged.var['highly_variable']]
    
    os.makedirs(sentence_output_basedir_hvg, exist_ok=True)
    hvg_file_path = os.path.join(sentence_output_basedir_hvg, "global_hvgs.txt")
    with open(hvg_file_path, 'w') as f:
        for gene in hvg_list:
            f.write(f"{gene}\n")
    
    print(f"质量控制完成! 最终: {adata_merged.n_obs} spots, {adata_merged.n_vars} genes")
    print(f"成功识别出 {len(hvg_list)} 个全局高变基因。")
    print(f"HVG列表已保存至: {hvg_file_path}")
    
    return hvg_list

In [3]:
def anndata_to_sentence_files_hvg(
    sample,
    hvg_list: pd.Index,
    output_basedir: str,
    n_top_genes_in_sentence: int = 50
):
    """
    为单个HESTSample对象的所有spots生成独立的基因句子文件，但仅限于HVG。

    参数:
        sample (HESTSample): 要处理的样本对象。
        hvg_list (pd.Index): 全局高变基因列表。
        output_basedir (str): 保存所有句子文件夹的根目录。
        n_top_genes_in_sentence (int): 每个句子包含的基因数量。
    """
    # 为当前样本创建独立的输出目录
    sample_sentence_dir = os.path.join(output_basedir, f"{sample.sample_id}_sentences_hvg")
    os.makedirs(sample_sentence_dir, exist_ok=True)
    
    adata = sample.adata
    
    # 筛选AnnData，使其只包含高变基因，这样可以大大加速后续处理
    # .copy() 是为了避免修改原始adata对象
    adata_hvg = adata[:, adata.var_names.isin(hvg_list)].copy()
    
    if adata_hvg.n_vars == 0:
        print(f"警告: 样本 {sample.sample_id} 中没有找到任何全局HVGs，已跳过。")
        return

    obs_df = adata_hvg.obs.copy()
    gene_names_hvg = adata_hvg.var_names.to_numpy()
    
    # 检查并获取坐标
    if 'pxl_col_in_fullres' not in obs_df.columns or 'pxl_row_in_fullres' not in obs_df.columns:
        obs_df['pxl_col_in_fullres'] = adata_hvg.obsm['spatial'][:, 0]
        obs_df['pxl_row_in_fullres'] = adata_hvg.obsm['spatial'][:, 1]
        
    # 遍历每个spot并生成文件
    for i in tqdm(range(adata_hvg.n_obs), desc=f"处理 {sample.sample_id} (HVG)", leave=False):
        spot_info = obs_df.iloc[i]
        row_coord = int(round(spot_info['pxl_row_in_fullres']))
        col_coord = int(round(spot_info['pxl_col_in_fullres']))
        sentence_filename = os.path.join(sample_sentence_dir, f"{sample.sample_id}_{row_coord}_{col_coord}.txt")
        
        if os.path.exists(sentence_filename):
            continue

        # 获取当前spot在HVG子集上的表达向量
        expression_vector = adata_hvg.X[i].toarray().flatten() if issparse(adata_hvg.X) else np.array(adata_hvg.X[i]).flatten()
            
        # 在HVG子集内排序
        sorted_indices_hvg = np.argsort(expression_vector)[-1::-1]
        
        # 选择前 N 个基因（现在无需过滤管家基因，因为它们大概率不是HVG）
        top_genes = gene_names_hvg[sorted_indices_hvg[:n_top_genes_in_sentence]]
        
        # 写入文件
        with open(sentence_filename, 'w') as f:
            f.write(" ".join(top_genes))

In [4]:
# --- 1. 设置和数据加载 ---
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
# 新的输出目录，以区分
sentence_output_basedir_hvg = "/cwStorage/nodecw_group/jijh/hest_sentences_human_all/"

print("初始化HEST数据集管理器...")
dataset = HESTDataset(data_dir=data_dir)
brain_samples = dataset.get_samples(species="Homo sapiens")
print(f"找到 {len(brain_samples)} 个样本待处理。")
print("-" * 50)


初始化HEST数据集管理器...
找到 649 个样本待处理。
--------------------------------------------------


In [5]:
from collections import Counter
import random

def report_varname_duplicates(samples):
    """
    对每个 HESTSample 中的 AnnData，检查 var_names 内部的重复项并打印出来。
    参数:
        samples (list): HESTSample 对象列表，要求每个 sample.adata 已经加载。
    """
    any_dup = False
    
    for sample in tqdm(samples, desc="检查基因名重复"):
        # 确保数据已加载
        if sample.adata is None:
            sample.load_st_data(lazy=True)
        adata = sample.adata
        if adata is None:
            print(f"[跳过] 样本 {sample.sample_id} 无法加载 AnnData。")
            continue

        names = list(adata.var_names)
        counts = Counter(names)
        # 找出在同一个 AnnData 里出现超过一次的基因名
        dups = [gene for gene, cnt in counts.items() if cnt > 1]
        
        if dups:
            any_dup = True
            print(f"\n⚠️ 样本 {sample.sample_id} 中共有 {len(dups)} 个重复基因名：")
            
            # 分析重复原因 - 随机选择几个重复基因进行详细分析
            sample_dups = random.sample(dups, min(3, len(dups)))
            
            for gene in sample_dups:
                duplicate_count = counts[gene]
                # 找到所有重复位置的索引
                duplicate_indices = [i for i, name in enumerate(names) if name == gene]
                
                print(f"  - 基因 '{gene}' 出现 {duplicate_count} 次，位置索引: {duplicate_indices}")
                
                # 检查这些重复位置的基因信息是否相同
                if hasattr(adata.var, 'gene_ids'):
                    gene_ids = [adata.var.iloc[idx]['gene_ids'] if 'gene_ids' in adata.var.columns else 'N/A' for idx in duplicate_indices]
                    print(f"    对应的基因ID: {gene_ids}")
                    if len(set(gene_ids)) > 1:
                        print(f"    ⚠️ 相同基因名对应不同的基因ID - 这可能是基因别名问题")
                    else:
                        print(f"    ✓ 相同基因名对应相同的基因ID - 这可能是数据处理过程中的重复")
            
            # 显示所有重复基因（如果列表太长，只显示前20个）
            print(f"  完整重复基因列表:")
            for g in dups[:20]:
                print(f"    - {g} (出现{counts[g]}次)")
            if len(dups) > 20:
                print(f"    ...（共 {len(dups)} 个重复基因，以上仅列出前 20 个）")

    if not any_dup:
        print("✅ 在所有样本内部均未发现重复的 var_names。")

# 执行检查
report_varname_duplicates(brain_samples)


检查基因名重复:   0%|          | 0/649 [00:00<?, ?it/s]

/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")



⚠️ 样本 TENX156 中共有 3 个重复基因名：
  - 基因 'TMSB15B' 出现 2 次，位置索引: [17777, 17780]
  - 基因 'TBCE' 出现 2 次，位置索引: [1793, 1794]
  - 基因 'HSPA14' 出现 2 次，位置索引: [8683, 8684]
  完整重复基因列表:
    - TBCE (出现2次)
    - HSPA14 (出现2次)
    - TMSB15B (出现2次)

⚠️ 样本 TENX155 中共有 3 个重复基因名：
  - 基因 'TBCE' 出现 2 次，位置索引: [1793, 1794]
  - 基因 'TMSB15B' 出现 2 次，位置索引: [17777, 17780]
  - 基因 'HSPA14' 出现 2 次，位置索引: [8683, 8684]
  完整重复基因列表:
    - TBCE (出现2次)
    - HSPA14 (出现2次)
    - TMSB15B (出现2次)

⚠️ 样本 TENX154 中共有 3 个重复基因名：
  - 基因 'HSPA14' 出现 2 次，位置索引: [8683, 8684]
  - 基因 'TBCE' 出现 2 次，位置索引: [1793, 1794]
  - 基因 'TMSB15B' 出现 2 次，位置索引: [17777, 17780]
  完整重复基因列表:
    - TBCE (出现2次)
    - HSPA14 (出现2次)
    - TMSB15B (出现2次)

⚠️ 样本 TENX153 中共有 3 个重复基因名：
  - 基因 'TMSB15B' 出现 2 次，位置索引: [17777, 17780]
  - 基因 'TBCE' 出现 2 次，位置索引: [1793, 1794]
  - 基因 'HSPA14' 出现 2 次，位置索引: [8683, 8684]
  完整重复基因列表:
    - TBCE (出现2次)
    - HSPA14 (出现2次)
    - TMSB15B (出现2次)


/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")



⚠️ 样本 TENX144 中共有 3 个重复基因名：
  - 基因 'HSPA14' 出现 2 次，位置索引: [8683, 8684]
  - 基因 'TMSB15B' 出现 2 次，位置索引: [17777, 17780]
  - 基因 'TBCE' 出现 2 次，位置索引: [1793, 1794]
  完整重复基因列表:
    - TBCE (出现2次)
    - HSPA14 (出现2次)
    - TMSB15B (出现2次)


KeyboardInterrupt: 

In [8]:

# --- 2. 计算全局高变基因 (HVGs) ---
# 这一步可能需要较长时间和较大内存，因为它会合并所有数据
# 如果内存不足，可以考虑从每个样本中随机抽取一部分spots来构建合并的adata

global_hvg_list = calculate_global_hvgs(
    samples_to_process=brain_samples,
    n_top_genes=5000, # 选择3000个HVGs是一个常见的SOTA实践
    sentence_output_basedir_hvg=sentence_output_basedir_hvg
)
print("-" * 50)



开始合并所有样本以计算全局HVGs...


加载并合并AnnData:   0%|          | 0/649 [00:00<?, ?it/s]

/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


发现重复基因名，正在处理样本中的 3 个重复基因...
发现重复基因名，正在处理样本中的 3 个重复基因...
发现重复基因名，正在处理样本中的 3 个重复基因...
发现重复基因名，正在处理样本中的 3 个重复基因...
发现重复基因名，正在处理样本中的 3 个重复基因...
发现重复基因名，正在处理样本中的 3 个重复基因...
所有样本已合并。总尺寸: (1362726, 223541)
开始进行SOTA质量控制和过滤...
过滤前: 1362726 spots, 223541 genes
过滤低表达基因数spots后: 1183467 spots
过滤高线粒体比例spots后: 1144234 spots
增强过滤: 基因必须在至少 572 个 spots 中表达。


/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:293: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


过滤低表达基因后: 56786 genes
进行标准化和对数变换...
计算高变基因 (使用 'cell_ranger' flavor)...


ValueError: Bin edges must be unique: Index([               -inf, 0.16111868948504884,  0.1649173682665769,
       0.16985546480562866, 0.17315772961015485, 0.17315772961015485,
       0.18426108504645128,  0.1873063350542543, 0.19279586579764546,
       0.19411118488957693,  0.2068762604079876, 0.20994393563809266,
        0.2177609491730924,  0.2364327113644291, 0.35356484399580135,
       0.37692686059190467, 0.40098744529756447,  0.5480210455120584,
        0.7871120597506805,   5.160039545545681,                 inf],
      dtype='float64').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [ ]:
'Apoe' in global_hvg_list

ad_mouse_genes = [
    "App",    # Amyloid precursor protein
    "Psen1",  # Presenilin 1
    "Psen2",  # Presenilin 2
    "Mapt",   # Microtubule-associated protein tau
    "Apoe",   # Apolipoprotein E
    "Trem2",  # Triggering receptor expressed on myeloid cells 2
    "Clu",    # Clusterin
    "Picalm", # Phosphatidylinositol binding clathrin assembly protein
    "Bin1",   # Bridging integrator 1
    "Cd33",   # Siglec-3
    "Abca7",  # ATP-binding cassette sub-family A member 7
    "Sorl1",  # Sortilin-related receptor 1
    "Hcar2",  # Hydroxycarboxylic acid receptor 2
    "Vps35",   # Vacuolar protein sorting 35
    "Csf1r", # Colony stimulating factor 1 receptor
    "Bace1",    # β-分泌酶 1，关键的 Aβ 生成酶:contentReference[oaicite:0]{index=0}
    "Ide",      # 胰岛素降解酶，可降解 Aβ:contentReference[oaicite:1]{index=1}
    "Mme",      # Neprilysin，同样参与 Aβ 清除:contentReference[oaicite:2]{index=2}
    "Cd2ap",    # 调控内吞与 Aβ 清除:contentReference[oaicite:3]{index=3}
    "Cr1",      # 补体受体 1，免疫相关:contentReference[oaicite:4]{index=4}
    "Adam10",   # α-分泌酶，非淀粉样途径关键酶:contentReference[oaicite:5]{index=5}
    "Plcg2",    # 液泡内吞与信号传导:contentReference[oaicite:6]{index=6}
    "Spi1",     # 转录因子，免疫调控筛选中被鉴定:contentReference[oaicite:7]{index=7}
    "Mef2c",    # 神经发育转录因子，功能筛选中被鉴定:contentReference[oaicite:8]{index=8}
    "Gab2",     # 信号转导分子，GWAS 关联:contentReference[oaicite:9]{index=9}
    "Abcc11",   # ATP 结合盒家族成员，筛选中发现:contentReference[oaicite:10]{index=10}
    "ATCG1",    # 筛选中发现的新的候选基因:contentReference[oaicite:11]{index=11}
    "Ank3",     # GWAS 关联基因:contentReference[oaicite:12]{index=12}
    "Ms4a6a",   # MS4A 家族，调控免疫与脂质代谢:contentReference[oaicite:13]{index=13}
    "Agfg2",    # 涉及细胞信号转导:contentReference[oaicite:14]{index=14}
    "Cyc1",     # 线粒体功能相关:contentReference[oaicite:15]{index=15}
    "Hla-dra",  # 抗原呈递分子:contentReference[oaicite:16]{index=16}
    "Meg3",     # 长链非编码 RNA，表达差异:contentReference[oaicite:17]{index=17}
    "Mt2a",     # 金属硫蛋白，抗氧化应激:contentReference[oaicite:18]{index=18}
    "Ncald",    # 钙调节蛋白家族:contentReference[oaicite:19]{index=19}
    "Neu1",     # 神经氨酸苷酶 1，细胞内吞作用:contentReference[oaicite:20]{index=20}
    "Psmc3",    # 泛素-蛋白酶体系统:contentReference[oaicite:21]{index=21}
    "Serpinb6", # 丝氨酸蛋白酶抑制剂:contentReference[oaicite:22]{index=22}
    "Sparc",     # 细胞外基质蛋白，病理重塑:contentReference[oaicite:23]{index=23}
]

# Test what ad_mouse_genes are in global_hvg_list
ad_mouse_genes_in_hvg = [gene for gene in ad_mouse_genes if gene in global_hvg_list]
print(f"在全局HVG列表中找到 {len(ad_mouse_genes_in_hvg)} 个小鼠AD相关基因：")
print(ad_mouse_genes_in_hvg)

在全局HVG列表中找到 11 个小鼠AD相关基因：
['Apoe', 'Trem2', 'Clu', 'Bin1', 'Cd33', 'Csf1r', 'Cd2ap', 'Spi1', 'Mef2c', 'Ncald', 'Sparc']


In [41]:
# Add the mouse AD genes to the global HVG list
global_hvg_list = global_hvg_list.union(ad_mouse_genes_in_hvg)

In [43]:
sentence_output_basedir_hvg

'/cwStorage/nodecw_group/jijh/hest_sentences/'

In [46]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading

# --- 3. 基于HVG列表，为每个样本生成句子文件 ---

print("开始批量生成基于HVG的基因句子文件...")

def process_single_sample(sample):
    """处理单个样本的函数"""
    try:
        # 确保数据已加载
        if sample.adata is None:
            sample.load_st_data(lazy=False)
            sample.adata = sample.adata.to_memory()
        if sample.adata is None:
            print(f"跳过样本 {sample.sample_id}，因为无法加载AnnData。")
            return
            
        anndata_to_sentence_files_hvg(
            sample=sample,
            hvg_list=global_hvg_list,
            output_basedir=sentence_output_basedir_hvg,
            n_top_genes_in_sentence=50
        )
        print(f"完成处理样本 {sample.sample_id}")
    except Exception as e:
        print(f"处理样本 {sample.sample_id} 时发生严重错误: {e}")

# 使用线程池执行器进行并行处理
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(process_single_sample, sample) for sample in brain_samples]
    
    # 等待所有任务完成并处理结果
    for future in tqdm(futures, desc="处理所有样本"):
        try:
            future.result()  # 这会等待任务完成并获取结果（如果有异常会抛出）
        except Exception as e:
            print(f"线程执行时发生错误: {e}")

print("\n所有基于HVG的基因句子已生成并保存到硬盘！")


开始批量生成基于HVG的基因句子文件...


处理 TENX88 (HVG):   0%|          | 0/5303 [00:00<?, ?it/s]

完成处理样本 TENX88


处理 TENX79 (HVG):   0%|          | 0/8462 [00:00<?, ?it/s]

完成处理样本 TENX79


...

处理 NCBI337 (HVG):   0%|          | 0/188 [00:00<?, ?it/s]

完成处理样本 NCBI337


处理 NCBI336 (HVG):   0%|          | 0/214 [00:00<?, ?it/s]

完成处理样本 NCBI336



KeyboardInterrupt



In [69]:

# --- 4. 验证结果 ---
# 对多个随机样本进行验证
import random

validation_results = []
num_validations = min(20, len(brain_samples))  # 最多验证1000次或所有样本数

print(f"\n--- 对 {num_validations} 个随机样本进行验证 ---")

for i in tqdm(range(num_validations), desc="验证样本"):
    # 随机选择一个样本
    random_int = random.randint(0, len(brain_samples) - 1)
    sample_id = brain_samples[random_int].sample_id
    sample_sentence_dir = os.path.join(sentence_output_basedir_hvg, f"{sample_id}_sentences_hvg")
    
    if os.path.exists(sample_sentence_dir) and os.listdir(sample_sentence_dir):
        sentence_files = os.listdir(sample_sentence_dir)
        if sentence_files:  # 确保有文件
            random_sentence_file = random.choice(sentence_files)
            
            try:
                with open(os.path.join(sample_sentence_dir, random_sentence_file), 'r') as f:
                    content = f.read()
                    genes_in_sentence = content.split()
                    
                    # 检查这些基因是否都在HVG列表中
                    are_all_hvgs = all(gene in global_hvg_list for gene in genes_in_sentence)
                    
                    validation_results.append({
                        'sample_id': sample_id,
                        'sentence_file': random_sentence_file,
                        'num_genes': len(genes_in_sentence),
                        'all_hvgs': are_all_hvgs,
                        'non_hvg_genes': [gene for gene in genes_in_sentence if gene not in global_hvg_list]
                    })
            except Exception as e:
                validation_results.append({
                    'sample_id': sample_id,
                    'sentence_file': random_sentence_file,
                    'num_genes': 0,
                    'all_hvgs': False,
                    'error': str(e)
                })

# 汇总验证结果
total_validations = len(validation_results)
successful_validations = sum(1 for r in validation_results if r.get('all_hvgs', False))
failed_validations = total_validations - successful_validations

print(f"\n=== 验证结果汇总 ===")
print(f"总验证次数: {total_validations}")
print(f"成功验证（所有基因都在HVG列表中）: {successful_validations}")
print(f"失败验证: {failed_validations}")
print(f"成功率: {successful_validations/total_validations*100:.2f}%")

# 显示一些失败的案例（如果有的话）
if failed_validations > 0:
    print(f"\n--- 失败案例分析 ---")
    failed_cases = [r for r in validation_results if not r.get('all_hvgs', False)]
    for i, case in enumerate(failed_cases[:5]):  # 只显示前5个失败案例
        print(f"失败案例 {i+1}:")
        print(f"  样本ID: {case['sample_id']}")
        print(f"  句子文件: {case['sentence_file']}")
        if 'error' in case:
            print(f"  错误: {case['error']}")
        else:
            print(f"  基因数量: {case['num_genes']}")
            print(f"  非HVG基因: {case['non_hvg_genes']}")

# 显示一些成功的案例
print(f"\n--- 成功案例示例 ---")
successful_cases = [r for r in validation_results if r.get('all_hvgs', False)]
if successful_cases:
    sample_case = random.choice(successful_cases)
    print(f"样本ID: {sample_case['sample_id']}")
    print(f"句子文件: {sample_case['sentence_file']}")
    print(f"基因数量: {sample_case['num_genes']}")
    print("✅ 所有基因都在全局HVG列表中")


--- 对 20 个随机样本进行验证 ---


KeyboardInterrupt: 

In [68]:
# --- 4. 验证结果 ---
# 随机选择一个样本目录进行检查
if brain_samples:
    # Randomlys select a sample to validate
    import random
    random_int = random.randint(0, len(brain_samples) - 1)
    first_sample_id = brain_samples[random_int].sample_id
    sample_sentence_dir = os.path.join(sentence_output_basedir_hvg, f"{first_sample_id}_sentences_hvg")
    
    if os.path.exists(sample_sentence_dir) and os.listdir(sample_sentence_dir):
        print(f"\n--- 验证样本 {first_sample_id} 的结果 ---")
        sentence_files = os.listdir(sample_sentence_dir)
        random_sentence_file = random.choice(sentence_files)
        
        with open(os.path.join(sample_sentence_dir, random_sentence_file), 'r') as f:
            content = f.read()
            genes_in_sentence = content.split()
            print(f"随机抽取的句子文件 ({random_sentence_file}):")
            print(content)
            print(f"句子中的基因数量: {len(genes_in_sentence)}")
            # 检查这些基因是否都在HVG列表中
            are_all_hvgs = all(gene in global_hvg_list for gene in genes_in_sentence)
            print(f"句子中的所有基因是否都在全局HVG列表中? {'是' if are_all_hvgs else '否'}")
    else:
        print("未能找到生成的句子文件进行验证。")


--- 验证样本 NCBI661 的结果 ---
随机抽取的句子文件 (NCBI661_10489_8313.txt):
Gm42418 Cst3 Apoe Fth1 Nsmf Slc1a2 Clu Camk2a Atp1a2 Malat1 Nrgn Mbp Mt1 Dbi Olfm1 Snap25 Plp1 Ptma Psd Ak5 Ptgds Nfix Slc17a7 Nptxr Camk2n1 6330403K07Rik Scd2 Diras2 Cldn5 Cryab Scn1b Htra1 Trf Dlgap3 Zbtb20 Ncdn Hist1h2bc Cnih2 Mlc1 Crym Rgs7bp Pex16 Adcy1 Ddn Rasl10b Ndfip1 Desi1 Kif5a Bcl11b Cplx2
句子中的基因数量: 50
句子中的所有基因是否都在全局HVG列表中? 是


# 采样——人类样本

In [1]:
# --- 安装必要的库 ---
# 请确保您已按照上述说明安装了 libvips 系统库
# pip install pyvips opencv-python scikit-image tqdm

import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.color import rgb2hed
from skimage.filters import threshold_otsu
from IPython.display import display, Image as IPImage

# # 导入 pyvips 库
# try:
#     import pyvips
# except ImportError:
#     print("错误: pyvips 库未找到。请按照单元格上方的说明进行安装。")
#     # 在 Notebook 中，我们可以用 raise 来停止执行
#     raise

# 使用 tqdm.notebook 可以在 Notebook 中显示更好看的进度条
from tqdm.auto import tqdm

os.chdir("/home1/jijh/diffusion_project/ADiffusion")
os.getcwd()


'/public/home/jijh/diffusion_project/ADiffusion'

In [5]:
# --- 安装必要的库 ---
# 请确保您已按照上述说明安装了 libvips 系统库
# pip install pyvips opencv-python scikit-image tqdm scanpy

import os
import time  # 导入时间库
import random
import scanpy as sc
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from scipy.sparse import issparse

# 导入您自定义的库
# 假设 HESTDataset 和 load_sample_data 函数已正确定义并在环境中可用
from src.pipeline.hest_loading import HESTDataset

# --- 辅助函数 ---

def format_time(seconds: float) -> str:
    """将秒数格式化为易读的“分-秒”字符串"""
    mins, secs = divmod(seconds, 60)
    return f"{int(mins)}分 {secs:.2f}秒"

def load_sample_data(sample, batch_key: str = 'sample_id'):
    """Helper function to load a single sample's data"""
    try:
        sample.load_st_data(lazy=False)
        if sample.adata is not None:
            sample.adata.obs_names = [f"{sample.sample_id}_{obs_name}" for obs_name in sample.adata.obs_names]
            sample.adata.obs[batch_key] = sample.sample_id
            return sample.adata.to_memory()
        return None
    except Exception as e:
        print(f"Error loading sample {sample.sample_id}: {e}")
        return None

# --- 核心功能函数 ---
def load_and_preprocess_data(
    data_dir: str, 
    cache_path: str,
    samples_to_exclude: set
) -> sc.AnnData:
    """
    执行数据加载、合并和预处理，并跳过指定的问题样本。
    如果缓存文件存在，则直接从缓存加载。
    """
    if os.path.exists(cache_path):
        print(f"✅ 成功从缓存加载预处理过的 AnnData 文件: {cache_path}")
        return sc.read_h5ad(cache_path)

    print("⚠️ 未能找到缓存文件，开始执行完整的预处理流程...")
    start_time = time.time()
    
    # 1. 加载所有样本
    print("初始化HEST数据集管理器...")
    dataset = HESTDataset(data_dir=data_dir)
    samples_to_process = dataset.get_samples(species="Homo sapiens")
    print(f"找到 {len(samples_to_process)} 个样本。")
    
    adatas = []
    skipped_count = 0
    for sample in tqdm(samples_to_process, desc="加载、过滤并合并AnnData"):
        # 【关键修改】 在加载前检查样本是否在排除列表中
        if sample.sample_id in samples_to_exclude:
            print(f"⚠️ 正在跳过问题样本: {sample.sample_id}")
            skipped_count += 1
            continue  # 跳过这个样本

        adata = load_sample_data(sample, 'sample_id')
        if adata is not None:
            if adata.var_names.duplicated().any():
                adata = adata[:, ~adata.var_names.duplicated()].copy()
            adatas.append(adata)
            
    if not adatas:
        raise ValueError("未能加载任何有效的AnnData对象。")

    print(f"\n成功加载 {len(adatas)} 个样本，跳过了 {skipped_count} 个问题样本。")

    # 2. 合并
    adata_merged = sc.concat(adatas, join='outer', uns_merge='unique')
    print(f"所有样本已合并。总尺寸: {adata_merged.shape}")

    # 3. 质量控制 (QC)
    print("开始进行质量控制和过滤...")
    adata_merged.var['mt'] = adata_merged.var_names.str.startswith(('MT-', 'mt-'))
    sc.pp.calculate_qc_metrics(adata_merged, qc_vars=['mt'], inplace=True, percent_top=None, log1p=False)
    
    sc.pp.filter_cells(adata_merged, min_genes=200)
    adata_merged = adata_merged[adata_merged.obs.pct_counts_mt < 20, :].copy()
    
    min_cells_threshold = int(adata_merged.n_obs * 0.001)
    print(f"增强过滤: 基因必须在至少 {min_cells_threshold} 个 spots 中表达。")
    sc.pp.filter_genes(adata_merged, min_cells=min_cells_threshold)

    # 清理 'in_tissue' 列以确保数据类型一致
    adata_merged.obs['in_tissue'] = adata_merged.obs['in_tissue'].fillna(0).astype(bool)

    print(f"预处理完成。最终尺寸: {adata_merged.shape}")
    
    # 4. 写入缓存
    print(f"正在将预处理结果缓存到: {cache_path}")
    os.makedirs(os.path.dirname(cache_path), exist_ok=True)
    adata_merged.write(cache_path)
    
    end_time = time.time()
    print(f"🕒 完整预处理耗时: {format_time(end_time - start_time)}")
    
    return adata_merged

def calculate_global_hvgs(
    adata_full: sc.AnnData,
    n_top_genes: int = 5000,
    subsample_fraction: float = 1.0,
    flavor: str = 'seurat_v3_paper',
    span: float = 0.5,
    output_dir: str = "output/hvgs",
    random_seed: int = 42,
    batch_key: str = 'sample_id'
):
    """
    在完整或二次采样的数据上计算HVG。专为调试和最终运行设计。
    """
    start_time = time.time() # 开始计时
    
    print("-" * 50)
    is_debug_run = subsample_fraction < 1.0
    run_mode = "⚡️ 快速调试模式 ⚡️" if is_debug_run else "🏁 最终运行模式 🏁"
    print(f"启动HVG计算: {run_mode}")

    adata_proc = adata_full
    if is_debug_run:
        print(f"对数据进行二次采样，使用 {subsample_fraction*100:.1f}% 的数据...")
        # 【关键优化】使用随机种子确保每次采样结果一致
        adata_proc = sc.pp.subsample(adata_full, fraction=subsample_fraction, copy=True, random_state=random_seed)
    else:
        adata_proc = adata_full.copy()

    print(f"处理数据维度: {adata_proc.shape}")

    if flavor not in ['seurat_v3', 'seurat_v3_paper']:
        print("进行标准化和对数变换...")
        sc.pp.normalize_total(adata_proc, target_sum=1e4)
        sc.pp.log1p(adata_proc)

    print(f"使用 flavor='{flavor}' 计算高变基因...")
    try:
        sc.pp.highly_variable_genes(
            adata_proc,
            n_top_genes=n_top_genes,
            flavor=flavor,
            span=span,
            batch_key=batch_key
        )
        hvg_list = adata_proc.var.index[adata_proc.var['highly_variable']]
        print(f"✅ 成功! 找到 {len(hvg_list)} 个高变基因。")

        if not is_debug_run:
            os.makedirs(output_dir, exist_ok=True)
            hvg_file_path = os.path.join(output_dir, "global_hvgs.txt")
            with open(hvg_file_path, 'w') as f:
                for gene in hvg_list: f.write(f"{gene}\n")
            print(f"✅ 全局HVG列表已保存至: {hvg_file_path}")
        
        end_time = time.time() # 结束计时
        print(f"🕒 HVG计算耗时: {format_time(end_time - start_time)}")
        return hvg_list

    except Exception as e:
        print(f"\n❌ 错误: 在HVG计算期间发生错误: {e}")
        # ... (错误处理建议保持不变) ...
        end_time = time.time()
        print(f"🕒 HVG计算耗时 (至失败点): {format_time(end_time - start_time)}")
        return None



In [ ]:
# --- 主执行流程 ---

# --- 1. 设置路径和种子 ---
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
output_basedir = "/cwStorage/nodecw_group/jijh/hest_sentences_human_all/"
processed_adata_cache_path = os.path.join(output_basedir, "cache", "adata_merged_preprocessed.h5ad")
subsample_fraction = 0.5
batch_key = None  # 用于HVG计算的批次键

# 【关键优化】设定全局随机种子以保证结果可复现
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
print(f"🌱 全局随机种子已设定为: {SEED}")

# --- 2. 加载或预处理数据 (高效) ---
adata_merged = load_and_preprocess_data(data_dir, processed_adata_cache_path)



🌱 全局随机种子已设定为: 42
✅ 成功从缓存加载预处理过的 AnnData 文件: /cwStorage/nodecw_group/jijh/hest_sentences_human_all/cache/adata_merged_preprocessed.h5ad


In [ ]:
# --- 1. 设置路径和种子 ---
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
output_basedir = "/cwStorage/nodecw_group/jijh/hest_sentences_human_all/"
processed_adata_cache_path = os.path.join(output_basedir, "cache", "adata_merged_preprocessed.h5ad")
subsample_fraction = 0.5
batch_key = None  # 用于HVG计算的批次键


# --- 3. 快速调试HVG计算 (在20%的数据上运行) ---
print("\n\n--- 第1步: 在中等规模数据上进行快速调试 ---")
global_hvg_list_debug = calculate_global_hvgs(
    adata_full=adata_merged,
    n_top_genes=5000,
    subsample_fraction=subsample_fraction,  # 使用20%的数据
    flavor='seurat_v3_paper',
    span=0.5,
    output_dir=output_basedir,
    random_seed=SEED,  # 传递种子
    batch_key=batch_key  # 如果需要，可以传递批次键
)

# --- 4. 根据调试结果决定下一步 ---
if global_hvg_list_debug is not None:
    print("\n\n--- 第2步: 调试成功！准备进行最终计算 ---")
    # ... (后续逻辑保持不变) ...
    
    # 如果需要自动进行最终计算，请取消下面的注释
    # print("\n开始在全部数据上进行最终计算...")
    # global_hvg_list_final = calculate_global_hvgs(
    #     adata_full=adata_merged,
    #     n_top_genes=5000,
    #     subsample_fraction=1.0,  # 使用100%的数据
    #     flavor='seurat_v3',
    #     span=0.5,
    #     output_dir=output_basedir,
    #     random_seed=SEED # 在最终运行时也传递种子，保持良好实践
    # )
    #
    # if global_hvg_list_final is not None:
    #     print("\n🎉 全部流程成功完成！🎉")
    # else:
    #     print("\n❌ 最终计算失败，请检查上面的错误日志。")

else:
    print("\n\n--- 调试失败 ---")
    print("❌ 快速调试运行失败。请根据上面的错误信息和建议进行调整。")

# 发现是batchkey的问题，检查是哪个样本出了问题

In [4]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading

# 假设 adata_merged 是您已经加载并预处理好的完整对象
print("--- 开始诊断，逐个检查每个样本 ---")

# 获取所有唯一的样本ID
all_batches = adata_merged.obs['sample_id'].unique()
problematic_batches = []

def test_single_batch(batch_id):
    """测试单个样本的函数"""
    try:
        print(f"\n正在测试样本: {batch_id}")
        # 提取单个样本的数据
        adata_batch = adata_merged[adata_merged.obs['sample_id'] == batch_id].copy()
        
        # # 模拟在单个样本上运行HVG计算的核心步骤
        # # 必须重新进行标准化和对数化，因为这些是在子集上计算的
        # sc.pp.normalize_total(adata_batch, target_sum=1e4)
        # sc.pp.log1p(adata_batch)
        
        # 在这个单个样本上运行HVG计算
        sc.pp.highly_variable_genes(
            adata_batch, 
            flavor='seurat_v3_paper', 
            n_top_genes=5000, 
            span=0.5
        )
        print(f"✅ 样本 {batch_id} 测试通过。")
        return batch_id, True, None
        
    except ValueError as e:
        # 如果捕获到我们一直在遇到的错误，就记录下来
        print(f"❌ 样本 {batch_id} 测试失败! 错误: {e}")
        return batch_id, False, str(e)

# 使用线程池执行器进行并行处理
with ProcessPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(test_single_batch, batch_id) for batch_id in all_batches]
    
    # 等待所有任务完成并收集结果
    for future in tqdm(futures, desc="检查样本"):
        try:
            batch_id, success, error = future.result()
            if not success:
                problematic_batches.append(batch_id)
        except Exception as e:
            print(f"线程执行时发生错误: {e}")

print("\n--- 诊断结束 ---")
if problematic_batches:
    print("发现以下问题样本:")
    for b in problematic_batches:
        print(f" - {b}")
    print("\n建议：您可以考虑在预处理步骤中将这些样本从分析中排除。")
else:
    print("✅ 所有样本均单独测试通过，问题可能更复杂。")

--- 开始诊断，逐个检查每个样本 ---

正在测试样本: TENX158
正在测试样本: TENX156
正在测试样本: TENX157

检查样本:   0%|          | 0/649 [00:00<?, ?it/s]


正在测试样本: TENX155
正在测试样本: TENX154


正在测试样本: TENX149
正在测试样本: TENX153

正在测试样本: TENX147
正在测试样本: TENX152






正在测试样本: TENX148
✅ 样本 TENX149 测试通过。

正在测试样本: TENX144
✅ 样本 TENX153 测试通过。
✅ 样本 TENX147 测试通过。
正在测试样本: TENX143


正在测试样本: NCBI884
✅ 样本 TENX154 测试通过。

正在测试样本: NCBI883
✅ 样本 TENX156 测试通过。

正在测试样本: NCBI882
✅ 样本 TENX155 测试通过。

正在测试样本: NCBI881
✅ 样本 TENX157 测试通过。✅ 样本 TENX158 测试通过。
✅ 样本 TENX148 测试通过。


正在测试样本: NCBI879
正在测试样本: NCBI880
正在测试样本: NCBI876


✅ 样本 NCBI884 测试通过。

正在测试样本: NCBI875
✅ 样本 NCBI883 测试通过。

正在测试样本: NCBI873
✅ 样本 NCBI881 测试通过。

正在测试样本: NCBI870
✅ 样本 NCBI882 测试通过。

正在测试样本: NCBI867
✅ 样本 TENX144 测试通过。

正在测试样本: NCBI866
✅ 样本 NCBI879 测试通过。

正在测试样本: NCBI865
✅ 样本 NCBI880 测试通过。

正在测试样本: NCBI864
✅ 样本 NCBI876 测试通过。

正在测试样本: NCBI861
✅ 样本 NCBI875 测试通过。

正在测试样本: NCBI860
✅ 样本 NCBI873 测试通过。

正在测试样本: NCBI859
✅ 样本 NCBI870 测试通过。

正在测试样本: NCBI858
✅ 样本 NCBI867 测试通过。

正在测试样本: NCBI857
✅ 样本 NCBI864 测试通过。

正在测试样本: NCBI856
✅ 样本 NCBI866 测试通过。

正在测试样本: MISC142
✅ 样本 NCBI865 测试通过。
✅ 样本 NCBI861 测试通过。
正在测试样本: MISC

# 移除问题样本，再执行一次

In [6]:
# --- 1. 设置路径、种子和参数 ---
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
output_basedir = "/cwStorage/nodecw_group/jijh/hest_sentences_human_all/"
processed_adata_cache_path = os.path.join(output_basedir, "cache", "adata_merged_preprocessed.h5ad")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
print(f"🌱 全局随机种子已设定为: {SEED}")

# 【关键修改】 定义需要排除的问题样本列表
SAMPLES_TO_EXCLUDE = {
    'TENX15', 'MEND16', 'MEND15', 'MEND14', 'MEND13', 'MEND12',
    'MEND11', 'MEND10', 'MEND9', 'MEND8', 'MEND7', 'MEND2',
    'MEND1', 'NCBI657', 'NCBI814'
}

# --- 2. 加载或预处理数据 (现在会排除问题样本) ---
adata_merged = load_and_preprocess_data(
    data_dir, 
    processed_adata_cache_path,
    samples_to_exclude=SAMPLES_TO_EXCLUDE
)


🌱 全局随机种子已设定为: 42
✅ 成功从缓存加载预处理过的 AnnData 文件: /cwStorage/nodecw_group/jijh/hest_sentences_human_all/cache/adata_merged_preprocessed.h5ad


In [9]:
print("\n\n--- 第1步: 在中等规模数据上进行快速调试 (带批次校正) ---")
global_hvg_list_debug = calculate_global_hvgs(
    adata_full=adata_merged,
    n_top_genes=5000,
    subsample_fraction=1,  # 使用50%的数据进行调试
    flavor='seurat_v3_paper',
    span=0.5,
    output_dir=output_basedir,
    random_seed=SEED,
    batch_key=None  # <--- 现在我们重新启用它！
)




--- 第1步: 在中等规模数据上进行快速调试 (带批次校正) ---
--------------------------------------------------
启动HVG计算: 🏁 最终运行模式 🏁
处理数据维度: (1144234, 49316)
使用 flavor='seurat_v3_paper' 计算高变基因...
✅ 成功! 找到 5000 个高变基因。
✅ 全局HVG列表已保存至: /cwStorage/nodecw_group/jijh/hest_sentences_human_all/global_hvgs.txt
🕒 HVG计算耗时: 2分 40.58秒


In [12]:
# Check if APOE is in the debug HVG list
'APOE' in global_hvg_list_debug

# Human AD-related genes (converted from mouse gene names)
ad_human_genes = [
    "APP",      # Amyloid precursor protein
    "PSEN1",    # Presenilin 1
    "PSEN2",    # Presenilin 2
    "MAPT",     # Microtubule-associated protein tau
    "APOE",     # Apolipoprotein E
    "TREM2",    # Triggering receptor expressed on myeloid cells 2
    "CLU",      # Clusterin
    "PICALM",   # Phosphatidylinositol binding clathrin assembly protein
    "BIN1",     # Bridging integrator 1
    "CD33",     # Siglec-3
    "ABCA7",    # ATP-binding cassette sub-family A member 7
    "SORL1",    # Sortilin-related receptor 1
    "HCAR2",    # Hydroxycarboxylic acid receptor 2
    "VPS35",    # Vacuolar protein sorting 35
    "CSF1R",    # Colony stimulating factor 1 receptor
    "BACE1",    # β-secretase 1, key Aβ generating enzyme
    "IDE",      # Insulin degrading enzyme, can degrade Aβ
    "MME",      # Neprilysin, also involved in Aβ clearance
    "CD2AP",    # Regulates endocytosis and Aβ clearance
    "CR1",      # Complement receptor 1, immune related
    "ADAM10",   # α-secretase, key enzyme in non-amyloidogenic pathway
    "PLCG2",    # Vacuolar endocytosis and signal transduction
    "SPI1",     # Transcription factor, identified in immune regulation screening
    "MEF2C",    # Neuronal development transcription factor, identified in functional screening
    "GAB2",     # Signal transduction molecule, GWAS associated
    "ABCC11",   # ATP binding cassette family member, found in screening
    "ANK3",     # GWAS associated gene
    "MS4A6A",   # MS4A family, regulates immunity and lipid metabolism
    "AGFG2",    # Involved in cell signal transduction
    "CYC1",     # Mitochondrial function related
    "HLA-DRA",  # Antigen presentation molecule
    "MEG3",     # Long non-coding RNA, expression differences
    "MT2A",     # Metallothionein, anti-oxidative stress
    "NCALD",    # Calcium regulatory protein family
    "NEU1",     # Neuraminidase 1, cellular endocytosis
    "PSMC3",    # Ubiquitin-proteasome system
    "SERPINB6", # Serine protease inhibitor
    "SPARC",    # Extracellular matrix protein, pathological remodeling
]

# Test what ad_mouse_genes are in global_hvg_list
ad_mouse_genes_in_hvg = [gene for gene in ad_mouse_genes if gene in global_hvg_list_debug]
print(f"在全局HVG列表中找到 {len(ad_mouse_genes_in_hvg)} 个小鼠AD相关基因：")
print(ad_mouse_genes_in_hvg)

在全局HVG列表中找到 0 个小鼠AD相关基因：
[]


In [13]:
global_hvg_list_debug

Index(['A1BG', 'A2ML1', 'AADACL2', 'AADACL3', 'AB019441.29', 'ABCA1', 'ABCA12',
       'ABCA4', 'ABCB4', 'ABCC11',
       ...
       '__ambiguous[NME1-NME2+NME2]', '__ambiguous[RNASEK+RNASEK-C17orf49]',
       '__ambiguous[RP11-1035H13.3+RPS15A]', '__ambiguous[RPL12P16+NBEAL1]',
       '__ambiguous[RPL17-C18orf32+RPL17]',
       '__ambiguous[RPL36A+RPL36A-HNRNPH2]', '__ambiguous[RPS10+RPS10-NUDT3]',
       '__ambiguous[ZNF90+CTC-260E6.2]', 'antisense_PROKR2',
       'antisense_SCRIB'],
      dtype='object', length=5000)

In [14]:
len(global_hvg_list_debug)

5000

# 原来基因名称没统一

In [17]:
# --- 1. 安装和导入必要的库 ---
import os
import sys  # 导入 sys 模块以修改路径
import time
import random
import scanpy as sc
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from scipy.sparse import issparse

# 导入您自定义的库
# 假设 HESTDataset 已在您的环境中正确定义
from src.pipeline.hest_loading import HESTDataset

# --- 2. 辅助函数 ---

def format_time(seconds: float) -> str:
    """将秒数格式化为易读的“分-秒”字符串"""
    mins, secs = divmod(seconds, 60)
    return f"{int(mins)}分 {secs:.2f}秒"

def filter_adata_by_canonical_genes(adata: sc.AnnData, canonical_genes: set) -> sc.AnnData:
    """
    使用权威基因列表严格过滤AnnData对象。
    """
    print("\n--- 开始基于权威列表进行基因名清洗和过滤 ---")
    num_before = adata.n_vars

    # 步骤 1: 对原始基因名进行基础修正（移除前缀和版本号）
    adata.var_names = [
        name.replace('GRCh38______', '').split('.')[0] 
        for name in adata.var_names
    ]
    
    # 步骤 2: 创建布尔掩码，只保留存在于权威列表中的基因
    keep_mask = adata.var_names.isin(list(canonical_genes))
    
    adata = adata[:, keep_mask].copy()
    num_after_filter = adata.n_vars
    print(f"基于权威列表过滤后，基因数量从 {num_before} 减少到 {num_after_filter}。")
    
    # 步骤 3: 对剩余的干净基因名进行唯一化处理
    print("对剩余的基因名进行唯一化处理...")
    adata.var_names_make_unique()
    
    print("--- 基因名清洗完成 ---\n")
    return adata


def load_and_preprocess_data(
    data_dir: str, 
    cache_path: str,
    samples_to_exclude: set,
    canonical_genes: set
) -> sc.AnnData:
    """
    执行数据加载、合并、基于权威列表的基因名清洗和预处理。
    """
    if os.path.exists(cache_path):
        print(f"✅ 成功从缓存加载预处理过的 AnnData 文件: {cache_path}")
        return sc.read_h5ad(cache_path)

    print("⚠️ 未能找到缓存文件，开始执行完整的预处理流程...")
    start_time = time.time()
    
    # ... (加载和排除样本的逻辑保持不变) ...
    dataset = HESTDataset(data_dir=data_dir)
    samples_to_process = dataset.get_samples(species="Homo sapiens")
    adatas = []
    skipped_count = 0
    for sample in tqdm(samples_to_process, desc="加载、过滤并合并AnnData"):
        if sample.sample_id in samples_to_exclude:
            skipped_count += 1
            continue
        adata = load_sample_data(sample, 'sample_id')
        if adata is not None:
            if adata.var_names.duplicated().any():
                adata = adata[:, ~adata.var_names.duplicated()].copy()
            adatas.append(adata)
    print(f"\n成功加载 {len(adatas)} 个样本，跳过了 {skipped_count} 个问题样本。")

    adata_merged = sc.concat(adatas, join='outer', uns_merge='unique')
    print(f"合并后，初始尺寸: {adata_merged.shape}")

    # 【关键步骤】使用权威列表进行清洗
    adata_merged = filter_adata_by_canonical_genes(adata_merged, canonical_genes)
    print(f"基因名清洗后，尺寸: {adata_merged.shape}")

    # ... (后续的QC、过滤和保存逻辑保持不变) ...
    print("开始进行质量控制和过滤...")
    adata_merged.var['mt'] = adata_merged.var_names.str.startswith(('MT-', 'mt-'))
    sc.pp.calculate_qc_metrics(adata_merged, qc_vars=['mt'], inplace=True, percent_top=None, log1p=False)
    sc.pp.filter_cells(adata_merged, min_genes=200)
    adata_merged = adata_merged[adata_merged.obs.pct_counts_mt < 20, :].copy()
    min_cells_threshold = int(adata_merged.n_obs * 0.001)
    print(f"增强过滤: 基因必须在至少 {min_cells_threshold} 个 spots 中表达。")
    sc.pp.filter_genes(adata_merged, min_cells=min_cells_threshold)
    if 'in_tissue' in adata_merged.obs.columns:
        adata_merged.obs['in_tissue'] = adata_merged.obs['in_tissue'].fillna(0).astype(bool)

    print(f"预处理完成。最终尺寸: {adata_merged.shape}")
    
    print(f"正在将预处理结果缓存到: {cache_path}")
    os.makedirs(os.path.dirname(cache_path), exist_ok=True)
    adata_merged.write(cache_path)
    
    end_time = time.time()
    print(f"🕒 完整预处理耗时: {format_time(end_time - start_time)}")
    
    return adata_merged

def calculate_global_hvgs(
    adata_full: sc.AnnData,
    n_top_genes: int = 5000,
    flavor: str = 'seurat_v3_paper',
    span: float = 0.8,
    output_dir: str = "output/hvgs",
    batch_key: str = 'sample_id'
):
    """在完整、干净的数据上计算HVG"""
    start_time = time.time()
    print("-" * 50)
    print("🏁 开始在完整、干净的数据上进行最终HVG计算 🏁")
    
    adata_proc = adata_full.copy()
    print(f"处理数据维度: {adata_proc.shape}")

    print(f"使用 flavor='{flavor}' 和 batch_key='{batch_key}' 计算高变基因...")
    try:
        sc.pp.highly_variable_genes(
            adata_proc, n_top_genes=n_top_genes, flavor=flavor,
            span=span, batch_key=batch_key
        )
        hvg_list = adata_proc.var.index[adata_proc.var['highly_variable']]
        print(f"✅ 成功! 找到 {len(hvg_list)} 个高变基因。")
        
        os.makedirs(output_dir, exist_ok=True)
        hvg_file_path = os.path.join(output_dir, "global_hvgs.txt")
        with open(hvg_file_path, 'w') as f:
            for gene in hvg_list: f.write(f"{gene}\n")
        print(f"✅ 全局HVG列表已保存至: {hvg_file_path}")
        
        end_time = time.time()
        print(f"🕒 HVG计算耗时: {format_time(end_time - start_time)}")
        return hvg_list

    except Exception as e:
        print(f"\n❌ 错误: 在HVG计算期间发生错误: {e}")
        return None



In [18]:
# --- 主执行流程 ---

# --- 1. 设置路径、种子和参数 ---
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
output_basedir = "/cwStorage/nodecw_group/jijh/hest_sentences_human_all/"
cache_dir = os.path.join(output_basedir, "cache")
os.makedirs(cache_dir, exist_ok=True)

adata_cache_path = os.path.join(cache_dir, "adata_preprocessed_canonical_v2.h5ad")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
print(f"🌱 全局随机种子已设定为: {SEED}")

SAMPLES_TO_EXCLUDE = {
    'TENX15', 'MEND16', 'MEND15', 'MEND14', 'MEND13', 'MEND12',
    'MEND11', 'MEND10', 'MEND9', 'MEND8', 'MEND7', 'MEND2',
    'MEND1', 'NCBI657', 'NCBI814'
}

# --- 2. 【关键修改】从本地 .py 文件导入权威基因列表 ---
try:
    # 将 cache 目录临时添加到 Python 的搜索路径中
    sys.path.insert(0, cache_dir)
    from human_gene_symbols import human_gene_symbols
    # 将导入的列表转换为集合(set)以获得 O(1) 的超快查找速度
    canonical_genes_set = set(human_gene_symbols)
    print(f"✅ 成功从本地 'human_gene_symbols.py' 文件加载 {len(canonical_genes_set)} 个权威基因符号。")
except ImportError:
    print(f"\n❌ 错误: 未能在 '{cache_dir}' 目录下找到 'human_gene_symbols.py' 文件。")
    print("请确保您已运行之前的脚本来生成此文件。")
    raise
except Exception as e:
    print(f"加载本地基因列表时出错: {e}")
    raise
finally:
    # 恢复原始的系统路径，避免潜在的副作用
    sys.path.pop(0)


# --- 3. 加载、清洗并预处理数据 ---
adata_merged = load_and_preprocess_data(
    data_dir, 
    adata_cache_path,
    samples_to_exclude=SAMPLES_TO_EXCLUDE,
    canonical_genes=canonical_genes_set
)



🌱 全局随机种子已设定为: 42
✅ 成功从本地 'human_gene_symbols.py' 文件加载 44199 个权威基因符号。
⚠️ 未能找到缓存文件，开始执行完整的预处理流程...


加载、过滤并合并AnnData:   0%|          | 0/649 [00:00<?, ?it/s]

/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/public/home/jijh/micromamba/envs/imgprocess/lib


成功加载 634 个样本，跳过了 15 个问题样本。
合并后，初始尺寸: (1351225, 220010)

--- 开始基于权威列表进行基因名清洗和过滤 ---
增强过滤: 基因必须在至少 997 个 spots 中表达。
预处理完成。最终尺寸: (997054, 30148)
正在将预处理结果缓存到: /cwStorage/nodecw_group/jijh/hest_sentences_human_all/cache/adata_preprocessed_canonical_v2.h5ad
🕒 完整预处理耗时: 20分 30.94秒


In [19]:
# --- 4. 在完整的干净数据上进行最终计算 ---
global_hvg_list = calculate_global_hvgs(
    adata_full=adata_merged,
    n_top_genes=5000,
    flavor='seurat_v3_paper',
    span=0.5,
    output_dir=output_basedir,
    batch_key='sample_id'
)

# --- 5. 最终结果 ---
if global_hvg_list is not None:
    print("\n\n🎉🎉🎉 全部流程成功完成！🎉🎉🎉")
    print("最终获得的高变基因列表的前10个为:")
    print(global_hvg_list[:10].to_list())
else:
    print("\n\n--- 最终运行失败 ---")
    print("❌ 请根据上面的错误信息进行检查。")

--------------------------------------------------
🏁 开始在完整、干净的数据上进行最终HVG计算 🏁
处理数据维度: (997054, 30148)
使用 flavor='seurat_v3_paper' 和 batch_key='sample_id' 计算高变基因...
✅ 成功! 找到 5000 个高变基因。
✅ 全局HVG列表已保存至: /cwStorage/nodecw_group/jijh/hest_sentences_human_all/global_hvgs.txt
🕒 HVG计算耗时: 2分 41.54秒


🎉🎉🎉 全部流程成功完成！🎉🎉🎉
最终获得的高变基因列表的前10个为:
['A2M', 'A2ML1', 'A4GALT', 'AACS', 'AADAC', 'AADAT', 'AAGAB', 'AAMDC', 'AAMP', 'AAR2']


In [23]:
# Check if APOE is in the debug HVG list
'APOE' in global_hvg_list


True

In [26]:

# Human AD-related genes (converted from mouse gene names)
ad_human_genes = [
    "APP",      # Amyloid precursor protein
    "PSEN1",    # Presenilin 1
    "PSEN2",    # Presenilin 2
    "MAPT",     # Microtubule-associated protein tau
    "APOE",     # Apolipoprotein E
    "TREM2",    # Triggering receptor expressed on myeloid cells 2
    "CLU",      # Clusterin
    "PICALM",   # Phosphatidylinositol binding clathrin assembly protein
    "BIN1",     # Bridging integrator 1
    "CD33",     # Siglec-3
    "ABCA7",    # ATP-binding cassette sub-family A member 7
    "SORL1",    # Sortilin-related receptor 1
    "HCAR2",    # Hydroxycarboxylic acid receptor 2
    "VPS35",    # Vacuolar protein sorting 35
    "CSF1R",    # Colony stimulating factor 1 receptor
    "BACE1",    # β-secretase 1, key Aβ generating enzyme
    "IDE",      # Insulin degrading enzyme, can degrade Aβ
    "MME",      # Neprilysin, also involved in Aβ clearance
    "CD2AP",    # Regulates endocytosis and Aβ clearance
    "CR1",      # Complement receptor 1, immune related
    "ADAM10",   # α-secretase, key enzyme in non-amyloidogenic pathway
    "PLCG2",    # Vacuolar endocytosis and signal transduction
    "SPI1",     # Transcription factor, identified in immune regulation screening
    "MEF2C",    # Neuronal development transcription factor, identified in functional screening
    "GAB2",     # Signal transduction molecule, GWAS associated
    "ABCC11",   # ATP binding cassette family member, found in screening
    "ANK3",     # GWAS associated gene
    "MS4A6A",   # MS4A family, regulates immunity and lipid metabolism
    "AGFG2",    # Involved in cell signal transduction
    "CYC1",     # Mitochondrial function related
    "HLA-DRA",  # Antigen presentation molecule
    "MEG3",     # Long non-coding RNA, expression differences
    "MT2A",     # Metallothionein, anti-oxidative stress
    "NCALD",    # Calcium regulatory protein family
    "NEU1",     # Neuraminidase 1, cellular endocytosis
    "PSMC3",    # Ubiquitin-proteasome system
    "SERPINB6", # Serine protease inhibitor
    "SPARC",    # Extracellular matrix protein, pathological remodeling
]

# Test what ad_mouse_genes are in global_hvg_list
ad_human_genes_in_hvg = [gene for gene in ad_human_genes if gene in global_hvg_list]
print(f"在全局HVG列表中找到 {len(ad_human_genes_in_hvg)} 个人AD相关基因：")
print(ad_human_genes_in_hvg)

在全局HVG列表中找到 10 个人AD相关基因：
['APOE', 'CLU', 'BIN1', 'CSF1R', 'BACE1', 'CD2AP', 'CR1', 'ANK3', 'AGFG2', 'CYC1']


In [29]:
global_human_hvg_list = global_hvg_list.union(ad_human_genes)

In [30]:
len(global_human_hvg_list)

5028

In [35]:
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from scipy.sparse import issparse
from concurrent.futures import ThreadPoolExecutor
from types import SimpleNamespace # 用于创建轻量级的模拟对象


# --- 辅助函数: 基因句子生成 (已更新，更健壮) ---
def anndata_to_sentence_files_hvg(
    sample,
    hvg_list: pd.Index,
    output_basedir: str,
    n_top_genes_in_sentence: int = 50
):
    """
    为单个样本对象的所有spots生成独立的基因句子文件。
    【已更新】: 增加了对缺失坐标(NaN)的健壮处理。
    """
    sample_sentence_dir = os.path.join(output_basedir, f"{sample.sample_id}_sentences_hvg")
    os.makedirs(sample_sentence_dir, exist_ok=True)
    
    if sample.adata is None:
        return
        
    adata = sample.adata
    adata_hvg = adata[:, adata.var_names.isin(hvg_list)].copy()
    
    if adata_hvg.n_vars == 0:
        return

    obs_df = adata_hvg.obs.copy()
    gene_names_hvg = adata_hvg.var_names.to_numpy()
    
    # 检查并获取坐标
    if 'pxl_col_in_fullres' not in obs_df.columns or 'pxl_row_in_fullres' not in obs_df.columns:
        if 'spatial' in adata_hvg.obsm:
            obs_df['pxl_col_in_fullres'] = adata_hvg.obsm['spatial'][:, 0]
            obs_df['pxl_row_in_fullres'] = adata_hvg.obsm['spatial'][:, 1]
        else:
            obs_df['pxl_col_in_fullres'] = range(adata_hvg.n_obs)
            obs_df['pxl_row_in_fullres'] = 0

    # 遍历每个spot并生成文件
    for i in range(adata_hvg.n_obs):
        spot_info = obs_df.iloc[i]
        
        row_val = spot_info['pxl_row_in_fullres']
        col_val = spot_info['pxl_col_in_fullres']

        # --- 【【【关键修复点】】】---
        # 检查坐标值是否为 NaN。如果是，则使用spot的索引作为文件名。
        if pd.isna(row_val) or pd.isna(col_val):
            # 使用 spot 在 AnnData 中的行索引作为唯一标识符
            # 这确保了即使坐标缺失，文件名依然是唯一的
            row_coord = i 
            col_coord = 0 # 可以设为0或其他固定值
            filename_suffix = f"{row_coord}_{col_coord}_no_coord"
        else:
            # 如果坐标有效，则正常转换为整数
            row_coord = int(round(row_val))
            col_coord = int(round(col_val))
            filename_suffix = f"{row_coord}_{col_coord}"

        sentence_filename = os.path.join(sample_sentence_dir, f"{sample.sample_id}_{filename_suffix}.txt")
        # --- 【修复结束】---

        # 获取表达向量并生成句子
        expression_vector = adata_hvg.X[i].toarray().flatten() if issparse(adata_hvg.X) else np.array(adata_hvg.X[i]).flatten()
        sorted_indices_hvg = np.argsort(expression_vector)[-1::-1]
        top_genes = gene_names_hvg[sorted_indices_hvg[:n_top_genes_in_sentence]]
        
        with open(sentence_filename, 'w') as f:
            f.write(" ".join(top_genes))

In [36]:
from tqdm.auto import tqdm
from concurrent.futures import ProcessPoolExecutor

print("--- 开始批量生成基因句子文件 ---")
start_time_sentences = time.time()

# 从OmiCLIP论文中获取参数
N_TOP_GENES_IN_SENTENCE = 50
# 定义输出目录
sentence_output_basedir = "/cwStorage/nodecw_group/jijh/hest_sentences_human_all/"

# 获取所有需要处理的、干净的样本ID列表
# adata_merged 是之前步骤生成的、已经过清洗和过滤的AnnData对象
valid_sample_ids = adata_merged.obs['sample_id'].unique().tolist()
print(f"将在 {len(valid_sample_ids)} 个有效样本上生成基因句子。")

# 定义一个辅助函数，用于并行处理
def process_single_sample_wrapper(sample_id):
    try:
        # 1. 从大的adata_merged对象中提取单个样本的AnnData
        adata_sample = adata_merged[adata_merged.obs['sample_id'] == sample_id].copy()
        
        # 2. 创建一个模拟的'sample'对象，以匹配处理函数的输入要求
        mock_sample = SimpleNamespace(sample_id=sample_id, adata=adata_sample)
        
        # 3. 调用核心处理函数
        # global_hvg_list 是之前步骤计算出的高变基因列表
        anndata_to_sentence_files_hvg(
            sample=mock_sample,
            hvg_list=global_hvg_list,
            output_basedir=sentence_output_basedir,
            n_top_genes_in_sentence=N_TOP_GENES_IN_SENTENCE
        )
    except Exception as e:
        print(f"处理样本 {sample_id} 时发生严重错误: {e}")

# 使用线程池进行高效的并行文件写入
# max_workers可以根据您的CPU核心数进行调整，16是一个比较合理的起始值
with ProcessPoolExecutor(max_workers=16) as executor:
    # 使用tqdm来显示处理进度
    list(tqdm(executor.map(process_single_sample_wrapper, valid_sample_ids), total=len(valid_sample_ids), desc="处理所有样本"))

end_time_sentences = time.time()
print(f"\n🎉🎉🎉 全部样本处理完成！🕒 总耗时: {format_time(end_time_sentences - start_time_sentences)}")
print(f"所有基因句子文件已保存到: {sentence_output_basedir}")

# --- 最终验证 ---
print("\n--- 开始进行结果验证 ---")
try:
    random_sample_id = random.choice(valid_sample_ids)
    sample_sentence_dir = os.path.join(sentence_output_basedir, f"{random_sample_id}_sentences_hvg")
    
    if os.path.exists(sample_sentence_dir) and os.listdir(sample_sentence_dir):
        sentence_files = os.listdir(sample_sentence_dir)
        random_sentence_file = random.choice(sentence_files)
        
        with open(os.path.join(sample_sentence_dir, random_sentence_file), 'r') as f:
            content = f.read()
            genes_in_sentence = content.split()
            
            print(f"随机抽样验证成功！样本 '{random_sample_id}' 的一个句子文件内容如下：")
            print(f"文件名: {random_sentence_file}")
            print(f"句子内容: '{content}'")
            print(f"句子中的基因数量: {len(genes_in_sentence)}")
            
            are_all_hvgs = all(gene in global_hvg_list for gene in genes_in_sentence)
            print(f"句子中的所有基因是否都在全局HVG列表中? {'✅ 是' if are_all_hvgs else '❌ 否'}")
    else:
        print(f"验证失败：未能找到样本 '{random_sample_id}' 的输出目录或目录为空。")
except Exception as e:
    print(f"验证过程中发生错误: {e}")

--- 开始批量生成基因句子文件 ---
将在 505 个有效样本上生成基因句子。


处理所有样本:   0%|          | 0/505 [00:00<?, ?it/s]


🎉🎉🎉 全部样本处理完成！🕒 总耗时: 3分 21.30秒
所有基因句子文件已保存到: /cwStorage/nodecw_group/jijh/hest_sentences_human_all/

--- 开始进行结果验证 ---
随机抽样验证成功！样本 'TENX30' 的一个句子文件内容如下：
文件名: TENX30_2160_3605.txt
句子内容: 'VEGFA-1 SPP1-1 CEBPD TIMP1-1 CTSB FOS B2M ITGB4-1 ANXA2 CEBPB HSPB1-1 DDIT4 NFKBIA-1 LIFR-1 NPM1-1 EMP3 CTNNA1 JUN-1 CCND2 HMGA1-1 CD276 NOTCH3-1 ITGB8-1 ITGA3-1 HSPA1A-1 CD74 SOX9-1 NUPR1-1 BCL6 CTNND1 ERRFI1 TP53-1 SPRY2-1 EGR3 ERBB2 FAT1 DNAJB1 COL11A1 PLCG2-1 WEE1-1 RAD51C-1 DAZAP1 RBX1-1 GADD45B GADD45A RHOB-1 CXCR4 BAMBI COL5A1 CBFB'
句子中的基因数量: 50
句子中的所有基因是否都在全局HVG列表中? ✅ 是
